In [1]:
import os
import sys
import datetime
from pathlib import Path
import logging
from main import SigmoidFitter, MeltomeAtlasHandler, setup_logging

import json
import numpy as np
import pandas as pd
import seaborn as sns
# for logfile in [f for f in os.listdir() if '.log' in f]:
#     print(logfile)
    # os.remove(logfile)
now = datetime.datetime.now()
timestamp_str = now.strftime("%Y-%m-%d_%H-%M")
setup_logging(f'./{timestamp_str}_testing.log', logging.DEBUG)

2025-12-10 17:02:30 - root - INFO - Logging to file: ./2025-12-10_17-02_testing.log


In [2]:
# SCRIPT_DIR = Path(__file__).resolve().parent
CWD = Path(os.getcwd()).resolve()
PROJECT_ROOT = CWD.parent.parent
print(PROJECT_ROOT)
file_path = PROJECT_ROOT / "Data" / "FLIP" / "splits" / "meltome" / "full_dataset.json"
output_path = f'./results_meltome_{timestamp_str}'
print(file_path)

C:\Users\alexa\Documents\PROHITS
C:\Users\alexa\Documents\PROHITS\Data\FLIP\splits\meltome\full_dataset.json


In [3]:
# Sample subset of Meltome
data_handler = MeltomeAtlasHandler(file_path, output_path, logging.DEBUG)
print(data_handler.data.shape)
df = data_handler.select_subset(100)
chunk0 = data_handler.data.iloc[:25]
data_handler.data = df 
print(data_handler.data.shape)

2025-12-10 17:02:30 - MeltomeAtlasHandler - INFO - Meltome Handler initialization
2025-12-10 17:02:30 - MeltomeAtlasHandler - DEBUG - DEBUG Mode
2025-12-10 17:02:32 - MeltomeAtlasHandler - INFO - Meltome data loaded from C:\Users\alexa\Documents\PROHITS\Data\FLIP\splits\meltome\full_dataset.json
2025-12-10 17:02:32 - MeltomeAtlasHandler - INFO - Meltome header ['proteinId', 'uniprotAccession', 'runName', 'meltingPoint', 'meltingBehaviour', 'sequence', 'quantNormMeltingPoint']
2025-12-10 17:02:32 - MeltomeAtlasHandler - INFO - Output directory: c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\results_meltome_2025-12-10_17-02
(221203, 7)
(2500, 7)


In [4]:
n = np.arange(20)
s = np.array_split(n, 10)
print(data_handler.data.shape[0], len(s), np.sum([len(i) for i in s]))

2500 10 20


In [5]:
test_data = pd.DataFrame(data_handler.data.iloc[0].meltingBehaviour)

In [6]:
test_data

,temperature,channel,fold_change
0,25.0,TMT126,1.000000
1,36.0,TMT127H,0.625354
2,30.0,TMT127L,0.712107
3,45.0,TMT128H,0.494478
4,39.0,TMT128L,0.574600
5,56.0,TMT129H,0.274403
6,50.0,TMT129L,0.362902
7,66.9,TMT130H,0.076361
8,59.6,TMT130L,0.108771
9,70.4,TMT131L,0.073697


In [7]:
t = SigmoidFitter()
print(t.get_statistics())
r = t.fit_curve(test_data.temperature.to_numpy(), test_data.fold_change.to_numpy())
print(t.get_statistics())

{'rmse': nan, 'r_squared': nan}
{'rmse': np.float64(0.053790785523586175), 'r_squared': np.float64(0.9660244340601158)}


In [8]:
x = t.get_melting_temp()
print(x)

42.49


c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: divide by zero encountered in divide
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


In [9]:
results0 = data_handler.process_chunk(chunk0)

2025-12-10 17:02:33 - MeltomeAtlasHandler - INFO - Chunk progress : 0 / 25, pid : A0A023PXQ4, specie : Saccharomyces cerevisiae lysate
2025-12-10 17:02:33 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:02:33 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9918, RMSE = 0.0280
2025-12-10 17:02:33 - SigmoidFitter - DEBUG - Fitted parameters: pl = -0.387047, a = 396.281525, b = 6.904978
2025-12-10 17:02:33 - MeltomeAtlasHandler - INFO - Chunk progress : 1 / 25, pid : A0A023T4K3, specie : Caenorhabditis elegans lysate
2025-12-10 17:02:33 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:02:33 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9867, RMSE = 0.0453
2025-12-10 17:02:33 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.052359, a = 719.635395, b = 19.066874
2025-12-10 17:02:33 - MeltomeAtlasHandler - INFO - Chunk progress : 2 / 25, pid : A0A023T778, specie : Mus musculus BMDC lysate
2025-12-10 17:02:33 - SigmoidFitter - DEBU

c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: overflow encountered in exp
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


In [10]:
results0_df = pd.DataFrame(results0)

In [11]:
data_handler.data

,proteinId,uniprotAccession,runName,meltingPoint,meltingBehaviour,sequence,quantNormMeltingPoint
34865,Q9LPD9_MCM2,Q9LPD9,Arabidopsis thaliana seedling lysate,40.821800,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MAGENSDNEPSSPASPSSAGFNTDQLPISTSQNSENFSDEEEAAVD...,NaN
36017,Q9SSM4_CHMP1B,Q9SSM4,Arabidopsis thaliana seedling lysate,43.568943,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MGNTDKLMNQIFDLKFTSKSLQRQSRKCEKEEKAEKLKVKKAIEKG...,NaN
36108,Q9SZ83_At4g09670,Q9SZ83,Arabidopsis thaliana seedling lysate,53.330266,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MATETQIRIGVMGCADIARKVSRAIHLAPNATISGVASRSLEKAKA...,NaN
21935,Q570C8_KAT5,Q570C8,Arabidopsis thaliana seedling lysate,43.977093,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MERAMERQKILLRHLNPVSSSNSSLKHEPSLLSPVNCVSEVSPMAA...,NaN
7064,O48917_SQD1,O48917,Arabidopsis thaliana seedling lysate,51.313361,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MAHLLSASCPSVISLSSSSSKNSVKPFVSGQTFFNAQLLSRSSLKG...,NaN
...,...,...,...,...,...,...,...
193899,SUCLG2,Q96I99,pTcells,53.564578,"[{'temperature': 37.0, 'fold_change': 0.999985...",MASPVAAQAGKLLRALALRPRFLAAGSQAVQLTSRRWLNLQEYQSK...,53.097300
215341,WDR55,Q9H6Y2,pTcells,51.905211,"[{'temperature': 37.0, 'fold_change': 0.999985...",MDRTCEERPAEDGSDEEDPDSMEAPTRIRDTPEDIVLEAPASGLAF...,52.661288
177423,RRBP1,Q9P2E9,pTcells,53.114765,"[{'temperature': 37.0, 'fold_change': 0.999985...",MDIYDTQTLGVVVFGGFMVVSAIGIFLVSTFSMKETSYEEALANQR...,51.694965
200008,TMED2,Q15363,pTcells,56.083833,"[{'temperature': 37.0, 'fold_change': 0.999985...",MVTLAELLVLLAALLATVSGYFVSIDAHAEECFFERVTSGTKMGLI...,55.441830


In [12]:
results0_df

,pid,runName,pl,a,b,rmse,r_squared,tm_pred,tm_flip
0,A0A023PXQ4,Saccharomyces cerevisiae lysate,-0.387047,396.281525,6.904978,0.0280,0.9918,52.99,52.403034
1,A0A023T4K3,Caenorhabditis elegans lysate,0.052359,719.635395,19.066874,0.0453,0.9867,37.96,37.962947
2,A0A023T778,Mus musculus BMDC lysate,0.106919,1749.919673,32.061066,0.0564,0.9812,54.99,54.425342
3,A0A061ACF5,Caenorhabditis elegans lysate,0.278508,295.604929,6.790961,0.0246,0.9868,49.46,49.459216
4,A0A061ACH4,Caenorhabditis elegans lysate,0.023551,724.433779,17.056474,0.0390,0.9914,42.59,42.593131
5,A0A061ACH8,Caenorhabditis elegans lysate,0.052800,559.097425,14.824896,0.0189,0.9977,38.00,37.999478
6,A0A061ACH9,Caenorhabditis elegans lysate,0.022312,602.364195,14.213085,0.0412,0.9878,42.52,42.517527
7,A0A061ACI3,Caenorhabditis elegans lysate,0.237867,660.532837,15.694724,0.0486,0.9727,43.89,43.892211
8,A0A061ACL3,Caenorhabditis elegans lysate,0.081819,934.223132,22.436277,0.0397,0.9909,41.97,41.973245
9,A0A061ACL6,Caenorhabditis elegans lysate,-1152.945257,224.780039,-3.236776,0.0587,0.9484,49.87,49.044775


In [15]:
results = data_handler.process(num_chunks=10)

2025-12-10 17:02:39 - MeltomeAtlasHandler - INFO - Starting curve fitting process
2025-12-10 17:02:39 - MeltomeAtlasHandler - DEBUG - Data split into 10 chunks
2025-12-10 17:02:39 - MeltomeAtlasHandler - INFO - Processing chunk 0 / 10 (size : 250)
2025-12-10 17:02:39 - MeltomeAtlasHandler - INFO - Chunk progress : 0 / 250, pid : Q9LPD9, specie : Arabidopsis thaliana seedling lysate
2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9660, RMSE = 0.0538
2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Fitted parameters: pl = -0.856702, a = 112.034857, b = 1.638446
2025-12-10 17:02:39 - MeltomeAtlasHandler - INFO - Chunk progress : 1 / 250, pid : Q9SSM4, specie : Arabidopsis thaliana seedling lysate
2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.7495, RMSE = 0.1295
2025-12-10 17:02:39 - Sigm

c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: divide by zero encountered in divide
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.024427, a = 278.576752, b = 6.574347
2025-12-10 17:02:39 - MeltomeAtlasHandler - INFO - Chunk progress : 8 / 250, pid : Q9FNJ5, specie : Arabidopsis thaliana seedling lysate
2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.8948, RMSE = 0.0704
2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.304397, a = 214.508729, b = 5.259971
2025-12-10 17:02:39 - MeltomeAtlasHandler - INFO - Chunk progress : 9 / 250, pid : Q9FJL3, specie : Arabidopsis thaliana seedling lysate
2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.8172, RMSE = 0.0931
2025-12-10 17:02:39 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.584152, a = 10468.146753, b = 209.496801
2025-12-10 17:02:39 - MeltomeAtlasHandler - INFO - Ch

c:\Users\alexa\Documents\PROHITS\Code\MeltingBehaviourCLI\main.py:61: RuntimeWarning: overflow encountered in exp
  y = (1 - pl) / (1 + np.exp(b - (a / x))) + pl


2025-12-10 17:03:28 - MeltomeAtlasHandler - ERROR - FAILURE (Fit): Q9W768 - Danio rerio Zenodo lysate at index 37104 failed to converge. Reason: Curve fitting failed to converge.
2025-12-10 17:03:28 - MeltomeAtlasHandler - INFO - Chunk progress : 96 / 250, pid : F1QUU0, specie : Danio rerio Zenodo lysate
2025-12-10 17:03:28 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:03:28 - SigmoidFitter - DEBUG - Fitting complete. R² = 0.9193, RMSE = 0.0496
2025-12-10 17:03:28 - SigmoidFitter - DEBUG - Fitted parameters: pl = 0.516781, a = 708.418122, b = 13.195806
2025-12-10 17:03:28 - MeltomeAtlasHandler - INFO - Chunk progress : 97 / 250, pid : F1QFF9, specie : Danio rerio Zenodo lysate
2025-12-10 17:03:28 - SigmoidFitter - DEBUG - Initial parameters: [0.1, 1, 0.1]
2025-12-10 17:03:29 - MeltomeAtlasHandler - ERROR - FAILURE (Fit): F1QFF9 - Danio rerio Zenodo lysate at index 3816 failed to converge. Reason: Curve fitting failed to converge.
2025-12-10 17:03:29 - Meltom

In [ ]:
print(len(data_handler.data), len(results))

102


In [30]:
pid, runName = results.loc[0][['pid', 'runName']]
print(pid, runName)
data_handler.data[(data_handler.data.uniprotAccession == pid) & (data_handler.data.runName == runName)]


Q9LPD9 Arabidopsis thaliana seedling lysate


,proteinId,uniprotAccession,runName,meltingPoint,meltingBehaviour,sequence,quantNormMeltingPoint
34865,Q9LPD9_MCM2,Q9LPD9,Arabidopsis thaliana seedling lysate,40.8218,"[{'temperature': 25.0, 'channel': 'TMT126', 'f...",MAGENSDNEPSSPASPSSAGFNTDQLPISTSQNSENFSDEEEAAVD...,NaN


In [31]:
results[results.notna().all(axis=1)]

,pid,runName,pl,a,b,rmse,r_squared,tm_pred,tm_flip
0,Q9LPD9,Arabidopsis thaliana seedling lysate,-0.856702,112.034857,1.638446,0.0538,0.9660,42.49,40.821800
1,Q9SSM4,Arabidopsis thaliana seedling lysate,0.428566,545.092885,14.457010,0.1295,0.7495,43.57,43.568943
2,Q9SZ83,Arabidopsis thaliana seedling lysate,-0.049042,412.385162,7.615664,0.0429,0.9839,53.49,53.330266
3,Q570C8,Arabidopsis thaliana seedling lysate,0.018151,308.330606,7.048141,0.0272,0.9940,43.98,43.977093
4,O48917,Arabidopsis thaliana seedling lysate,0.064301,739.948615,14.557849,0.0478,0.9863,51.31,51.313361
...,...,...,...,...,...,...,...,...,...
2393,Q9NVU0,pTcells,0.017946,906.727144,18.989404,0.0455,0.9872,47.84,47.841203
2394,Q96I99,pTcells,0.024509,3263.659626,60.979698,0.0385,0.9928,53.56,53.564578
2395,Q9H6Y2,pTcells,-0.007800,911.404506,17.538175,0.0191,0.9976,51.92,51.905211
2396,Q9P2E9,pTcells,0.217661,11897.867620,224.575340,0.1497,0.8816,53.11,53.114765


In [32]:
results_na = results[results.isna().any(axis=1)]
print(results_na.shape)

(329, 9)
